In [143]:
import requests 
from requests import get
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
from multiprocessing import Pool

## Henter url fra foretaksregisteret

In [144]:
url = "https://hotell.difi.no/api/csv/brreg/enhetsregisteret?"

In [145]:
df = pd.read_csv(url, sep=';', encoding='utf-8')
df = df[df.url.notna()]

In [146]:
urls = df['url'].unique().tolist()

In [147]:
urls

['www.mayasart.net',
 'www.ugandasbarn.org',
 'www.arnestad.org',
 'mikkelcappelensmith.com',
 'www.plussark.no',
 'www.pluss2.no',
 'palmera-i-bergen.no',
 'www.ebe-data.com',
 'www.soldier.no',
 'artofdesign.ltevensen.com',
 'www.artific.no',
 'www.01kommunikasjon.no',
 'www.travsport.no/Organisasjon/ForbundLag/Hedmark-Travforbund/Travlag/Grue-Travklubb/',
 'www.07.no',
 'www.getpersonal.no',
 'www.08aerosystem.se',
 'www.12x.no',
 'www.1aid.no']

In [148]:
def getContent(url):
    response = requests.get(f'http://{url}')
    if response.ok:
        return response.text
    response = requests.get(f'https://{url}')
    if response.ok:
        return response.text

In [149]:
from bs4 import BeautifulSoup
content = BeautifulSoup(getContent(urls[0]), 'html.parser')
body = content.find('body')
content = body.findChildren()

In [150]:
content_string = ''.join(str(content))

In [151]:
from html.parser import HTMLParser

class MLStripper(HTMLParser):
    def __init__(self):
        self.reset()
        self.strict = False
        self.convert_charrefs= True
        self.fed = []
    def handle_data(self, d):
        self.fed.append(d)
    def get_data(self):
        return ''.join(self.fed)

def strip_tags(html):
    s = MLStripper()
    s.feed(html)
    return s.get_data()

In [152]:
import re
content_string = strip_tags(content_string)

content_string = re.sub(r'[^\x00-\x7F]+',' ', content_string)
tags = ['\n','\t','\r','\xa0','word-wrap:break-word;', '.wordwrapfix','{','}']
for tag in tags:
    content_string = content_string.replace(tag, ' ')
    
#content_string = re.sub('  +', '', content_string)

In [153]:
content_string = content_string.strip()

In [155]:
content_string[:500]

'[          Meny          Mayasart     Akryl maleri    ttebladroser   Kurs   Info kurs   Messer og utstilling   Kontakt    Om meg:   Gjestebok   Link     Nettbutikk     Adventskalendar   Strikkegnomer   Hardangersaumkort    ttebladroser   Foto p  lerret   I am   I exsist   Original Maleri   Bunadskjorte posar                                 Mayasart                         Kunst og design                                Skapargleda i fargar p  lerret, ogs  med syn l og stikkepinnar. Trykk av  ttbl'